In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.head()

,from_address,to_address,value,gas,gas_price,block_timestamp,transaction_hash
0,0x1d80acdcae2257329557ae12c1d1fc21361def30,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,0,290427,12366879475,2024-01-25 04:04:59.000000 UTC,0xb865e943901c75e354a0996d7366d4eb8371927654f4...
1,0x1d80acdcae2257329557ae12c1d1fc21361def30,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,0,290427,12366879475,2024-01-25 04:04:59.000000 UTC,0xb865e943901c75e354a0996d7366d4eb8371927654f4...
2,0xc4c313291e03aed45259397ebc55a158bd2820a0,0x74c719a36ef628422213f7e8a5c6910fd4bcb25f,0,113374,10418920340,2024-01-25 04:16:47.000000 UTC,0xdb5f0289941bf9ee1d7f4c0501ad004043b2e11901f1...
3,0x55dcad916750c19c4ec69d65ff0317767b36ce90,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,0,529273,10488182632,2024-01-25 05:48:59.000000 UTC,0x80f8cfa70676b4500f99e833eee4a7dbdb3c5e056ded...
4,0x55dcad916750c19c4ec69d65ff0317767b36ce90,0xad3b67bca8935cb510c8d18bd45f0b94f54a968f,0,529273,10488182632,2024-01-25 05:48:59.000000 UTC,0x80f8cfa70676b4500f99e833eee4a7dbdb3c5e056ded...
